### Configuração para rodar no Goolge Colab
<a href="https://colab.research.google.com/github/caesarcc/python-tcc-url-fakenews-check/blob/main/jupyter/classificacao_passo02_treino_avaliacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Abrir no Colab"/></a>  
O treinamento no Colab levou pouco menos de uma hora, enquanto no computador local indicou que levaria mais de 2 dias.   
As primeiras célucas devem ser executadas para instalar a arquitetura de transformers e liberar o acesso ao drive do Colab.  
O arquivo .csv de entrada pode ser enviado por upload.

In [ ]:
!pip install -q transformers
!pip install -q wandb
!nvidia-smi

     |████████████████████████████████| 4.0 MB 28.6 MB/s 
     |████████████████████████████████| 77 kB 8.1 MB/s 
     |████████████████████████████████| 6.6 MB 59.4 MB/s 
     |████████████████████████████████| 895 kB 71.2 MB/s 
     |████████████████████████████████| 596 kB 29.1 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Treino e Avaliação do Modelo de Classificação

In [2]:
# Importação de bibliotecas utilizadas no treino e avaliação
import os
import torch
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score, precision_recall_fscore_support, matthews_corrcoef
from transformers.file_utils import is_torch_available
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from IPython.display import display
%matplotlib inline

### Carrega dados processados no passo 1

In [5]:
pd.set_option("display.max_rows", 50, 'display.max_colwidth', 200)
dados_processado = pd.read_csv('../dados/fakebr_corpus_processado.csv', sep = ',')
dados_processado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7074 entries, 0 to 7073
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Unnamed: 0             7074 non-null   int64 
 1   texto                  7074 non-null   object
 2   classe                 7074 non-null   int64 
 3   texto_limpo            7074 non-null   object
 4   texto_processado       7074 non-null   object
 5   qtde_texto_limpo       7074 non-null   int64 
 6   qtde_texto_processado  7074 non-null   int64 
dtypes: int64(4), object(3)
memory usage: 387.0+ KB


In [7]:
dados = dados_processado[['classe','texto_processado']]
dados_processado[['classe','texto','texto_processado','qtde_texto_processado']].sample(n=3)

,classe,texto,texto_processado,qtde_texto_processado
1730,0,"Na última semana, dois advogados contratados pelo ex-ministro Antonio Palocci para negociar delação premiada com a Lava Jato se reuniram com procuradores da República, no QG da força-tarefa, em Cu...","último semana , advogar contratar ex-ministro Antonio Palocci negociar delação premiar Lava Jato reunir procurador República , QG força-tarefa , Curitiba . pouco quilômetros dali , ser Polícia Fed...",139
23,0,"Mulher morre esfaqueada durante briga com esposa do ex-marido em Itaporanga\nVítima de 40 anos chegou a ser socorrida, mas morreu no hospital. Suspeita de 26 anos fugiu e não foi encontrada.\n\nUm...","Mulher morrer esfaquear durante brigar esposo ex-marido ItaporangaVítima 40 ano chegar socorrido , morrer hospital . Suspeita 26 ano fugir encontrar . mulher 40 ano morrer esfaquear esposo ex-mari...",92
2906,0,"Maia reafirma lealdade a Temer e se coloca como opção para a Presidência em duas ou três eleições. Presidente da Câmara dos Deputados foi entrevistado pelo jornalista Roberto Dávila, para a GloboN...","Maia reafirmar lealdade Temer colocar opção Presidência eleição . Presidente Câmara Deputados entrevistar jornalista Roberto Dávila , GloboNews . entrevisto jornalista Roberto Dávila , GloboNews ,...",84


### Geração de seed  
Com esta rotina consigo garantir a reprodução dos resultados mesmo que o ambiente for reiniciado
Aplicável às libs random, numpy e torch

In [9]:
RANDOM_SEED = 42
def garantir_reprodutividade(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

garantir_reprodutividade(RANDOM_SEED)

### Carregando o modelo pré-treinado BERTimbau

In [ ]:
model_name = "neuralmind/bert-base-portuguese-cased"
# carregando o modelo
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
# carregar tokenizador
tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=False)
#Conforme mensagem o erro é esperado pois o modelo BertForSequence... está sendo inicializado por um BertForPreTraining.
# - This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

### Separando treino e validação

In [17]:
dados_treino, dados_teste = train_test_split(dados, test_size=0.2, random_state=RANDOM_SEED)
dados_validacao, dados_teste = train_test_split(dados_teste, test_size=0.5, random_state=RANDOM_SEED)
display(f"Treinamento: {dados_treino.shape}, Teste: {dados_teste.shape}, Validação: {dados_validacao.shape}")

'Treinamento: (5659, 2), Teste: (708, 2), Validação: (707, 2)'

### Tokenização e geração dos tensores

In [18]:
# máximo de tokens por frase
TAMANHO_MAXIMO = 400
encodings_treino = tokenizer(dados_treino['texto_processado'].to_list(), truncation=True, padding=True, max_length=TAMANHO_MAXIMO)
encodings_teste = tokenizer(dados_teste['texto_processado'], truncation=True, padding=True, max_length=TAMANHO_MAXIMO)
encodings_validacao = tokenizer(dados_validacao['texto_processado'], truncation=True, padding=True, max_length=TAMANHO_MAXIMO)


NameError: name 'tokenizer' is not defined

In [ ]:
class TorchDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# Converte as listas tokenizadas um dataset de tensonres
dataset_treino = TorchDataset(encodings_treino, classes_traino)
dataset_validacao = TorchDataset(encodings_validacao, classes_validacao)

### Melhoria do Treinamento do modelo BERTimbau (fine-tuning)

In [ ]:
# Calcula todas métricas possíveis para analisar posteriormente
def calcula_metricas(pred):
  classes = pred.label_ids
  predicoes = pred.predictions.argmax(-1)
  precision, recall, f1, _ = precision_recall_fscore_support(classes, predicoes, average='binary')
  return {
      'accuracy': accuracy_score(classes, predicoes),
      'f1': f1,
      'precision': precision,
      'recall': recall,
      'mcc': matthews_corrcoef(classes, predicoes),
      # cálcula o coeficiente kappa (nível de concordância ou reprodutibilidade)
      #'kappa': cohen_kappa_score(classes, predicoes),
  }

### Hiperparâmetros confome monografia e documentação do HuggingFace

In [ ]:
# Caminho google drive: drive/MyDrive/PUC/TCC/modelos/bertimbau_avaliar_noticias
CAMINHO_MODELO = "../modelos/bertimbau_avaliar_noticias"
hiper_parametros = TrainingArguments(
    output_dir="",
    overwrite_output_dir=True,
    num_train_epochs=10,             
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=50,                 
    weight_decay=0.01,               
    evaluation_strategy='no',
    logging_dir='./logs',
    report_to="wandb"            
)

trainer = Trainer(
    model=model,
    args=hiper_parametros,
    train_dataset=dataset_treino,
    eval_dataset=dataset_validacao,
    tokenizer=tokenizer,
    compute_metrics=calcula_metricas
)

In [ ]:
# Treina o modelo
%%wandb
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5659
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1770


Step,Training Loss
500,0.197700
1000,0.039800
1500,0.010200


Saving model checkpoint to drive/MyDrive/Colab Notebooks/modelos/bertimbau_avaliar_noticias/checkpoint-500
Configuration saved in drive/MyDrive/Colab Notebooks/modelos/bertimbau_avaliar_noticias/checkpoint-500/config.json
Model weights saved in drive/MyDrive/Colab Notebooks/modelos/bertimbau_avaliar_noticias/checkpoint-500/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/Colab Notebooks/modelos/bertimbau_avaliar_noticias/checkpoint-500/tokenizer_config.json
Special tokens file saved in drive/MyDrive/Colab Notebooks/modelos/bertimbau_avaliar_noticias/checkpoint-500/special_tokens_map.json
Saving model checkpoint to drive/MyDrive/Colab Notebooks/modelos/bertimbau_avaliar_noticias/checkpoint-1000
Configuration saved in drive/MyDrive/Colab Notebooks/modelos/bertimbau_avaliar_noticias/checkpoint-1000/config.json
Model weights saved in drive/MyDrive/Colab Notebooks/modelos/bertimbau_avaliar_noticias/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/

TrainOutput(global_step=1770, training_loss=0.0703791468830432, metrics={'train_runtime': 1898.8986, 'train_samples_per_second': 14.901, 'train_steps_per_second': 0.932, 'total_flos': 5089169060535000.0, 'train_loss': 0.0703791468830432, 'epoch': 5.0})

### Validação do Modelo tunado

In [ ]:
metricas = trainer.evaluate()
acc = metricas['eval_accuracy']
f1 = metricas['eval_f1']
precision = metricas['eval_precision']
recall = metricas['eval_recall']
mcc = metricas['eval_mcc']
kappa = metricas['eval_kappa']

***** Running Evaluation *****
  Num examples = 1415
  Batch size = 64


In [ ]:
metricas

{'epoch': 5.0,
 'eval_accuracy': 0.9901060070671378,
 'eval_f1': 0.990084985835694,
 'eval_kappa': 0.9802119647191617,
 'eval_loss': 0.053305864334106445,
 'eval_mcc': 0.9802158812529587,
 'eval_precision': 0.9886845827439887,
 'eval_recall': 0.9914893617021276,
 'eval_runtime': 35.4465,
 'eval_samples_per_second': 39.919,
 'eval_steps_per_second': 0.649}

In [ ]:
model.save_pretrained(f"{CAMINHO_MODELO}/best_model")
tokenizer.save_pretrained(f"{CAMINHO_MODELO}/tokenizer")

Configuration saved in drive/MyDrive/PUC/TCC/modelos/bertimbau_avaliar_noticias/best_model/config.json
Model weights saved in drive/MyDrive/PUC/TCC/modelos/bertimbau_avaliar_noticias/best_model/pytorch_model.bin


### Validando o modelo

In [ ]:
plt.plot(history['train_acc'], label='train accuracy')
plt.plot(history['val_acc'], label='validation accuracy')

plt.title('Training history')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.ylim([0, 1])

print(classification_report(y_test, y_pred, target_names=class_names))

def show_confusion_matrix(confusion_matrix):
  hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
  hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
  hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
  plt.ylabel('True sentiment')
  plt.xlabel('Predicted sentiment');

cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
show_confusion_matrix(df_cm)

### Salvando o melhor modelo

In [ ]:
model.save_pretrained("/modelos/fake_url_bertimbau/melhor_modelo")